In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.metrics import silhouette_score, mean_squared_error

In [2]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

In [3]:
session.set_keyspace('stock_data')

In [8]:
# ✅ ดึง.... stock_price
rows_fin = session.execute("""
    SELECT symbol, date, open, high, low, close, volume FROM stock_price ALLOW FILTERING
""")
df_technical = pd.DataFrame(rows_fin, columns=['symbol', 'date', 'open', 'high', 'low', 'close', 'volume'])
df_technical = df_technical.dropna()

# ✅ ดึง..... candlestick_data
rows_price = session.execute("""
    SELECT symbol, time, close_price, high_price, low_price, open_price, value, volume FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price, columns=['symbol', 'time', 'close_price', 'high_price', 'low_price', 'open_price', 'value', 'volume'])
df_price['time'] = pd.to_datetime(df_price['time'])
df_price = df_price.sort_values(['symbol', 'time'])

In [11]:
df_technical['date'] = pd.to_datetime(df_technical['date'], errors='coerce')
df_technical = df_technical[['symbol', 'date', 'open', 'high', 'low', 'close', 'volume']]
df_technical = df_technical.sort_values(['symbol', 'date']).reset_index(drop=True)


In [ ]:
symbol = '24CS'
df_stock = df_technical[df_technical['symbol'] == symbol]


In [13]:
from datetime import datetime, timedelta

end_date = df_stock['date'].max()
start_date = end_date - timedelta(days=150)
df_stock = df_stock[(df_stock['date'] >= start_date) & (df_stock['date'] <= end_date)].reset_index(drop=True)


In [ ]:
close_prices = df_stock['close'].values

# เรียกใช้ฟังก์ชันเดิมที่คุณมี
# ฟังก์ชัน detect_head_and_shoulders ยังไม่ได้ถูกนิยามใน notebook นี้
# คุณต้องนิยามฟังก์ชันนี้ก่อนใช้งาน เช่น:

def detect_head_and_shoulders(prices):
    # ตัวอย่างโค้ดอย่างง่าย (placeholder)
    # ควรแทนที่ด้วยอัลกอริทึมจริงสำหรับตรวจจับแพทเทิร์น head and shoulders
    return []

patterns = detect_head_and_shoulders(close_prices)


In [24]:
df_candle = df_price[df_price['symbol'] == symbol]

if not df_candle.empty:
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_candle['time'],
        open=df_candle['open_price'],
        high=df_candle['high_price'],
        low=df_candle['low_price'],
        close=df_candle['close_price'],
        name='Candlestick'
    ))

    fig.update_layout(
        title=f"Candlestick: {symbol}",
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=700
    )

    fig.show()
else:
    print(f"No data for symbol: {symbol}")
